In [223]:
from importlib import reload
import tsm; reload(tsm)
import _keys; reload(_keys)
import blizzard; reload(blizzard)
import config; reload(config)
import kvstore; reload(kvstore)
import snapshot; reload(snapshot)
import elemental_arbitrage; reload(elemental_arbitrage)
import bliz_tsm_join; reload(bliz_tsm_join)

<module 'bliz_tsm_join' from 'C:\\Users\\Med\\Desktop\\wow\\bliz_tsm_join.py'>

In [224]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [225]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [226]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [227]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [216]:
from blizzard import item_search_names

In [228]:
item_info("flask of the frost wyrm")

{'id': 46376,
 'name': 'flask of the frost wyrm',
 'num_auctions': 47,
 'quantity': 360,
 'weight_sell': {'gold': 172.89, 'copper': 1728948.2722222223},
 'avg_sell': {'gold': 85.29, 'copper': 852927.4468085107},
 'max': {'gold': 230.0, 'copper': 2300000},
 'p80': {'gold': 210.0, 'copper': 2100000},
 'p50': {'gold': 39.47, 'copper': 394650},
 'p20': {'gold': 26.31, 'copper': 263100},
 'wp80': {'gold': 210.0, 'copper': 2100000},
 'wp50': {'gold': 210.0, 'copper': 2100000},
 'wp20': {'gold': 39.47, 'copper': 394650},
 'min': {'gold': 10.45, 'copper': 104498},
 'realm_market_value': {'gold': 10.84, 'copper': 108372},
 'realm_historical': {'gold': 12.87, 'copper': 128748},
 'region_historical': {'gold': 9.8, 'copper': 98017},
 'region_avg_sale_price': {'gold': 8.63, 'copper': 86285},
 'sale_pct': 57,
 'sold_per_day': 207,
 'headroom': -153,
 'market_skew_pct': -15.83,
 'auction_skew_pct': -3.57}

In [218]:
from elemental_arbitrage import print_elemental_arbitrage_report

print_elemental_arbitrage_report(tsm_ah, items)

SKIP        eternal-life
SKIP        eternal-earth
SKIP        eternal-air
SKIP        eternal-fire
YES          BS eternal-water (unit profit: 43251,headroom: 273)
SKIP        eternal-shadow
SKIP        primal-life
SKIP        primal-earth
SKIP        primal-air
SKIP        primal-fire
SKIP        primal-water
SKIP        primal-shadow
SKIP        essence-eternal
SKIP        essence-cosmic
SKIP        essence-nether
SKIP        essence-planar
SKIP        essence-astral
SKIP        essence-magic
SKIP        essence-mystic


In [219]:
tsm_ah[items.get_id("Earthsiege Diamond")]

{'auctionHouseId': 351,
 'itemId': 41334,
 'petSpeciesId': None,
 'minBuyout': 250000,
 'quantity': 69,
 'marketValue': 281595,
 'historical': 334995,
 'numAuctions': 35,
 'region': {'regionId': 13,
  'itemId': 41334,
  'petSpeciesId': None,
  'quantity': 62,
  'marketValue': 301114,
  'avgSalePrice': 248656,
  'salePct': 51,
  'soldPerDay': 32,
  'historical': 297282}}

In [220]:
default_tsm = {
    'auctionHouseId': 0,
    'itemId': 0,
    'petSpeciesId': None,
    'minBuyout': 0,
    'quantity': 0,
    'marketValue': 0,
    'historical': 0,
    'numAuctions': 0,
    'region': {
        'regionId': 0,
        'itemId': 0,
        'petSpeciesId': None,
        'quantity': 0,
        'marketValue': 0,
        'avgSalePrice': 0,
        'salePct': 0,
        'soldPerDay': 0,
        'historical': 0,
    }
}

def normalize_tsm(tsm_items, bliz_items):
    meta = [
        {
            "id": h["data"]["id"],
            "name": h["data"]["name"]["en_US"],
            "quality": h["data"]["quality"]["name"]["en_US"],
            "tsm": tsm_items.get(h["data"]["id"], default_tsm),

        }
        for h in bliz_items
    ]

    return [
        {
            "id": h["id"],
            "name": h["name"], 
            "quality": h["quality"],
            # The 1e4 is to convert the value into gold
            "market": h["tsm"]["marketValue"] / 1e4,
            "auction": h["tsm"]["minBuyout"] / 1e4,
            "historical": h["tsm"]["region"]["historical"] / 1e4,
            "sale_rate": h["tsm"]["region"]["salePct"]/100,
            "daily_sold": h["tsm"]["region"]["soldPerDay"],
        }
        for h in meta
    ]

herbs = normalize_tsm(items, item_query(blizzard_static, {"item_subclass.name.en_US": "Herb"}))
minerals = normalize_tsm(items, item_query(blizzard_static, {"item_subclass.name.en_US": "Metal & Stone"}))

NameError: name 'blizzard_static' is not defined

In [ ]:
found_sholazar = {"adder's": 3*20+15, "goldclover": 2*20+9, "frost lotus": 3, "deadnettle": 20+3, "tiger lily": 2*20+2, "crystallized life": 10*5+3}
rates_per_minute = {k: v/50 for (k, v) in found_sholazar.items()}
rates_per_minute